In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels.api as sm
import seaborn as sns
import os
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy.stats import boxcox 

# Load & Process Data

In [ ]:
prec_data = pd.read_csv(os.path.join('data', 'prec-Mainland-raw.csv'))
prec_data = prec_data.melt(id_vars=["year"], var_name="month_str", value_name="prec")
prec_data.head()

month_map = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6,
             'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}

prec_data["month"] = prec_data["month_str"].map(month_map)

# Build datetime column (use first day of month as convention)
prec_data["date"] = pd.to_datetime(dict(year=prec_data["year"], 
                                       month=prec_data["month"], 
                                       day=1))
prec_data.head()

In [ ]:
inverse_month_map = {v:k for k,v in month_map.items()}
inverse_month_map

In [ ]:
# Taken from https://www.ipma.pt/en/oclima/series.longas/?loc=Mainland&type=raw
temp_data = pd.read_csv(os.path.join('data', 'temp-Mainland-raw.csv'))
temp_data['month'] = temp_data['date'].str.extract(r'([0-9]{2})')
temp_data['year'] = temp_data['date'].str.extract(r'([0-9]{4})')
temp_data['month'] = pd.to_numeric(temp_data['month'])
temp_data['year'] = pd.to_numeric(temp_data['year'])
temp_data['date'] = pd.to_datetime(temp_data['date'], format='%m/%Y')
temp_data.head()

In [ ]:
full_data = pd.merge(temp_data, prec_data[["date", "prec"]], 
                  on="date", how="inner")
full_data['month_str'] = full_data['month'].map(inverse_month_map)

full_data['tdiff'] = full_data['tmax'] - full_data['tmin']

full_data.head()

In [ ]:
start, end = "1995-01-01", "2020-01-01"

In [ ]:
selected_data = full_data[full_data['date'].between(start, end)]

# Plot time Series

In [ ]:
def plot_time_series(df, date_col, var_col, ma_window=None):
    # 1. Set up the plot
    fig, ax = plt.subplots(figsize=(12, 6))

    # 2. Plot using the new 'date' column for the x-axis
    ax.plot(df[date_col], df[var_col], marker='o', linestyle='-')
    # 3. Optionally plot moving average
    if ma_window is not None and ma_window > 1:
        ma_series = df[var_col].rolling(window=ma_window).mean()
        ax.plot(df[date_col], ma_series, color='red', linewidth=2,
                label=f'{ma_window}-period MA')
        
    # 3. Format the date axis for clarity ✨
    # Set the major locator to find the start of each year
    ax.xaxis.set_major_locator(mdates.YearLocator(base=5))
    # Set the format of the major labels to show just the year (e.g., "2023")
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

    # To add ticks for every 3 months, you can use a minor locator
    ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=6))

    # 4. Add labels and a grid
    ax.set_title(var_col)
    ax.set_xlabel('Date')
    ax.set_ylabel('Value')
    ax.grid(True, which='major', alpha=0.6)
    ax.grid(True, which='minor', alpha=0.2)

    plt.tight_layout()
    plt.show()


In [ ]:
plot_time_series(selected_data, 'date', 'tmed', ma_window=12)

In [ ]:
plot_time_series(selected_data, 'date', 'tmax', ma_window=12)

In [ ]:
plot_time_series(selected_data, 'date', 'tmin', ma_window=12)

In [ ]:
plot_time_series(selected_data, 'date', 'prec', ma_window=12)

# Feature Engineering

In [ ]:
selected_data.head()

In [ ]:
diff_month = (selected_data.groupby(['year', 'month']).apply(lambda g: pd.Series({
        'date': g['date'].min(),
        'tdiff': g['tmax'].max() - g['tmin'].min()
    }))).reset_index(drop=True)

diff_month['date_str'] = [str(date) for date in diff_month['date']]
diff_month['month'] = diff_month['date_str'].str.extract(r'\-([0-9]{2})\-')
diff_month['year'] = diff_month['date_str'].str.extract(r'([0-9]{4})')
diff_month['month'] = pd.to_numeric(diff_month['month'])
diff_month['year'] = pd.to_numeric(diff_month['year'])

In [ ]:
diff_month

# Exploring new Feature

In [ ]:
plot_time_series(diff_month, 'date', 'tdiff', ma_window=12)

## Log Feat

In [ ]:
diff_month['log_diff'] = np.log(diff_month['tdiff'])

In [ ]:
plot_time_series(diff_month, 'date', 'log_diff', ma_window=12)

## Seasonal Plot

In [ ]:
temp_data = diff_month[['date','year','month', 'tdiff', 'log_diff']]
temp_data = temp_data.set_index('date')

In [ ]:
def seasonal_plot(df, col, title):
    _, ax = plt.subplots(figsize=(16, 8))
    sm.graphics.tsa.month_plot(df[col], ylabel='col', ax=ax)
    ax.set_title(title)
    ax.set_xlabel("Month")
    plt.show()

In [ ]:
seasonal_plot(temp_data, 'tdiff', "Seasonal Subseries Plot: Temperature Range (Max - Min)")

In [ ]:
seasonal_plot(temp_data, 'log_diff', "Seasonal Subseries Plot: Log Temperature Range (Max - Min)")

## Box Cox

In [ ]:
bc, lambda_ = boxcox(temp_data['tdiff'])
print(f"Estimated Lambda: {lambda_}")

plot_time_series(pd.DataFrame.from_dict({
    "date": list(temp_data.index),
    "box_cox": bc
}), 'date', 'box_cox', ma_window=12)

In [ ]:
bc, lambda_ = boxcox(temp_data['log_diff'])
print(f"Estimated Lambda: {lambda_}")

plot_time_series(pd.DataFrame.from_dict({
    "date": list(temp_data.index),
    "box_cox": bc
}), 'date', 'box_cox', ma_window=12)

## Linear Regression

In [ ]:
import statsmodels.formula.api as smf 

diff_month['time'] = diff_month['date'].dt.to_period('M').astype(int) 

# fir linear regression model
fit = smf.ols(formula='tdiff ~ time', data=diff_month).fit()
print(fit.summary())

In [ ]:
residuals = fit.resid
# Resíduos: detrended time series
plt.subplot(1, 1, 1)
plt.plot(diff_month['date'], residuals, color='blue')
plt.title("Temperature Range Detrended")
plt.ylabel("Residuals")
plt.xlabel("Time")

plt.show()

In [ ]:
import statsmodels.formula.api as smf 

diff_month['time'] = diff_month['date'].dt.to_period('M').astype(int) 

# fir linear regression model
fit = smf.ols(formula='tdiff ~ time', data=diff_month).fit()
print(fit.summary())

residuals = fit.resid
# Resíduos: detrended time series
plt.subplot(1, 1, 1)
plt.plot(diff_month['date'], residuals, color='blue')
plt.title("Temperature Range Detrended")
plt.ylabel("Residuals")
plt.xlabel("Time")

plt.show()

## Difference Operator

In [ ]:
tdiff_series = temp_data['tdiff']
tdiff_diff = tdiff_series.diff(12)
tdiff_diff

In [ ]:
plt.plot(temp_data.index, tdiff_diff, color='#ff9900')
plt.title("Temperature Range Differenced")
plt.ylabel("Temperature Range")
plt.xlabel("date")

In [ ]:
diff12 = tdiff_diff.to_frame()
diff12['year_month'] = pd.to_datetime(diff12.index)
diff12

In [ ]:
diff12 = tdiff_diff.to_frame()
diff12['year_month'] = pd.to_datetime(diff12.index)

x = diff12[['year_month','tdiff']]
x = x.set_index('year_month')

seasonal_plot(x, 'tdiff', "Seasonal Plot: tdiff diff 12")


## STL Decomposition

In [ ]:
from statsmodels.tsa.seasonal import STL     ## to STL decomposition

# STL with periodic seasonal component
stl_1 = STL(temp_data['log_diff'], seasonal=len(temp_data), robust=True).fit()

# STL with seasonal window of 13
stl_2 = STL(temp_data['log_diff'], seasonal=13, robust=True).fit()

In [ ]:
print("Periodic Season")
stl_1.plot()

In [ ]:
print("13 Season")
stl_2.plot()

In [ ]:
# STL with periodic seasonal component
stl_1 = STL(temp_data['tdiff'], seasonal=len(temp_data), robust=False).fit()

# STL with seasonal window of 13
stl_2 = STL(temp_data['tdiff'], seasonal=13, robust=False).fit()

stl_1.plot()
stl_2.plot()

In [ ]:

from statsmodels.stats.diagnostic import het_arch
het_arch(stl_1.resid) # Not heteroskedascity

## Lag Plots

In [ ]:
def lag_plot_grid(ts, lags=12, title="Lag Plots"):
    """Create grid of lag plots"""
    fig, axes = plt.subplots(3, 4, figsize=(15, 10))
    fig.suptitle(title, fontsize=16)
    
    for i in range(lags):
        row = i // 4
        col = i % 4
        
        # Create lagged series
        lagged = ts.shift(i+1)
        
        # Remove NaN values
        mask = ~(np.isnan(ts) | np.isnan(lagged))
        x = ts[mask]
        y = lagged[mask]
        
        # Scatter plot
        axes[row, col].scatter(y, x, alpha=0.9, s=10, color="steelblue", edgecolor="black")
        axes[row, col].set_title(f'Lag {i+1}')
        axes[row, col].set_ylabel('X(t)')
        axes[row, col].set_xlabel(f'X(t-{i+1})')
        axes[row, col].grid(True, alpha=0.3)

        # Compute correlation
        corr = np.corrcoef(y, x)[0, 1]
        axes[row, col].text(
            0.05, 0.95,
            f"r = {corr:.3f}",
            transform=axes[row, col].transAxes,
            fontsize=12,
            verticalalignment="top",
            bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.5)
        )

        # Fit LOWESS
        smoothed = lowess(x, y, frac=0.3)  # frac controls smoothing
        axes[row, col].plot(smoothed[:,0], smoothed[:,1], color="red", linewidth=1.5)

    plt.tight_layout()
    plt.show()


In [ ]:
lag_plot_grid(temp_data['tdiff'], title="Lag Plots: Tdiff")
lag_plot_grid(stl_1.resid, title="Lag Plots: Tdiff - Remainder")

In [ ]:
lag_plot_grid(temp_data['log_diff'], title="Lag Plots: Log Tdiff")
lag_plot_grid(stl_1.resid, title="Lag Plots: Log Tdiff - Remainder")